In [1]:
import os

import tensorflow as tf
from tensorflow import keras

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

In [3]:
train_labels.shape

(1000,)

In [4]:
train_images.shape

(60000, 28, 28)

In [5]:
train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [6]:
train_images.shape

(1000, 784)

In [7]:
def create_model():
    model = keras.models.Sequential([keras.layers.Dense(512,activation='relu',input_shape=(784,)),
                             keras.layers.Dropout(0.2),
                             keras.layers.Dense(10)]
                           )
    model.compile(optimizer='adam',
                 loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])
    return model

In [8]:
model = create_model()

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [11]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

In [12]:
checkpoint_dir

'training_1'

In [15]:
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True,verbose=1)

In [16]:
model.fit(train_images,train_labels,validation_data=(test_images,test_labels),callbacks=[cp_callback],epochs=10)

Epoch 1/10
28/32 [=========================>....] - ETA: 0s - loss: 1.1974 - accuracy: 0.6540
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 15ms/step - loss: 1.1281 - accuracy: 0.6670 - val_loss: 0.7129 - val_accuracy: 0.7900
Epoch 2/10
24/32 [=====================>........] - ETA: 0s - loss: 0.4316 - accuracy: 0.8750
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 0.4245 - accuracy: 0.8750 - val_loss: 0.5641 - val_accuracy: 0.8210
Epoch 3/10
24/32 [=====================>........] - ETA: 0s - loss: 0.2711 - accuracy: 0.9388
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 0.2904 - accuracy: 0.9280 - val_loss: 0.4612 - val_accuracy: 0.8610
Epoch 4/10
23/32 [====================>.........] - ETA: 0s - loss: 0.1912 - accuracy: 0.9552
Epoch 00004: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 

In [21]:
model = create_model()
loss,acc = model.evaluate(test_images,test_labels,verbose = 1)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 [==============================] - 0s 1ms/step - loss: 2.3269 - accuracy: 0.1100
Untrained model, accuracy: 11.00%


In [22]:
model.load_weights(checkpoint_path)

In [23]:
loss,acc = model.evaluate(test_images,test_labels,verbose = 1)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 [==============================] - 0s 1ms/step - loss: 0.4086 - accuracy: 0.8730
Untrained model, accuracy: 87.30%


In [24]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [28]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1901 - accuracy: 0.6560
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4308 - accuracy: 0.8780
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2725 - accuracy: 0.9340
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2104 - accuracy: 0.9450
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1608 - accuracy: 0.9670
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [29]:
new_model = tf.keras.models.load_model('saved_model/my_model')

In [30]:
new_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1750 - accuracy: 0.6710
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4245 - accuracy: 0.8790
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2925 - accuracy: 0.9320
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2105 - accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1557 - accuracy: 0.9630


In [32]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [33]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4196 - accuracy: 0.8640
Restored model, accuracy: 86.40%
